In [53]:
# | default_exp classes.DomoPage


In [54]:
# | exporti
from fastcore.basics import patch_to
from dataclasses import dataclass, field

import asyncio
import httpx

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoUser as dmdu
import domolibrary.utils.DictDot as util_dd

import domolibrary.routes.page as page_routes


In [55]:
# | export
@dataclass
class DomoPage:
    id: str
    title: str = None
    parent_page_id: str = None
    top_page_id: str = None
    auth: dmda.DomoAuth = field(default = None , repr = False)
    owners: list = field(default_factory=list)
    cards: list = field(default_factory=list)
    collections: list = field(default_factory=list)
    children: list = field(default_factory=list)
    is_locked : bool = None

    def display_url(self):
        return f"https://{self.auth.domo_instance}.domo.com/page/{self.id}"

In [56]:
# | exporti
@patch_to(DomoPage)
async def _get_domo_users(self, user_ls: [dict]):
    import domolibrary.classes.DomoUser as dmu
    import domolibrary.classes.DomoGroup as dmg
    tasks = []

    for user in user_ls:
        if user.get('type') =='USER':
            tasks.append(dmu.DomoUser.get_by_id(user_id=user.get('id') or user.get('userId'), auth=self.auth))
        if user.get('type') == 'GROUP':
            tasks.append(dmg.DomoGroup.get_by_id(group_id=user.get('id') or user.get('groupId'), auth=self.auth))

    return await asyncio.gather( *tasks)


@patch_to(DomoPage, cls_method=True)
async def _from_bootstrap(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    domo_page = cls(id=dd.id, title=dd.title, auth=auth)

    if isinstance(dd.owners, list) and len(dd.owners) > 0:
        domo_page.owners = await domo_page._get_domo_users(page_obj.get('owners'))

    if isinstance(dd.children, list) and len(dd.children) > 0:
        domo_page.children = [
            cls._from_bootstrap(page_obj=child_dd, auth=auth)
            for child_dd in dd.children
            if child_dd.type == "page"
        ]

        [print(other_dd) for other_dd in dd.children
            if other_dd.type != "page"]

    return domo_page


#### sample _from_bootstrap_routes

In [57]:
import os
import domolibrary.routes.bootstrap as bootstrap_routes

auth = dmda.DomoFullAuth(
    domo_instance='domo-community',
    domo_password = os.environ['DOJO_PASSWORD'],
    domo_username= os.environ['DOMO_USERNAME'])

res = await bootstrap_routes.get_bootstrap_pages(auth = auth)

page_obj = res.response[10]
await DomoPage._from_bootstrap(page_obj, auth = auth)

DomoPage(id='790951325', title='Coronavirus (COVID-19) Dashboard', parent_page_id=None, top_page_id=None, owners=[DomoUser(id='1345408759', title='Major Domo Support Specialist', department='Owner', display_name='Alexis Lorenz (DataMaven)', email_address='DataMaven42@gmail.com', role_id=1, phone_number='518-496-4710', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=1590156374000, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=[], children=[<coroutine object DomoPage._from_bootstrap at 0x7fbdfe7e2c70>, <coroutine object DomoPage._from_bootstrap at 0x7fbdfe7e3060>, <coroutine object DomoPage._from_bootstrap at 0x7fbdfe7e2c00>], is_locked=None)

In [58]:
# | exporti

@patch_to(DomoPage, cls_method=True)
async def _from_content_stacks_v3(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):
    # import domolibrary.classes.DomoCard as dc

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(
        id=dd.id,
        title=dd.title,
        parent_page_id=dd.page.parentPageId,
        collections=dd.collections,
        auth=auth
    )

    if dd.page.owners and len(dd.page.owners) > 0:
        pg.owners = await pg._get_domo_users(page_obj.get('page').get('owners'))

    # if dd.cards and len(dd.cards) > 0:
    #     pg.cards = await asyncio.gather(
    #         *[dc.DomoCard.get_from_id(id=card.id, auth=auth) for card in dd.cards])

    return pg


@patch_to(DomoPage, cls_method=True)
async def get_by_id(cls: DomoPage,
                    page_id: str,
                    auth: dmda.DomoAuth,
                    return_raw: bool = False, 
                    debug_api: bool = False):

    res = await page_routes.get_page_by_id(auth=auth, page_id=page_id, debug_api=debug_api)

    if return_raw:
        return res
        
    if not res.status == 200:
        return

    pg = await cls._from_content_stacks_v3(page_obj=res.response, auth=auth)

    return pg


#### sample implementation of get_from_id

In [59]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = await DomoPage.get_by_id(page_id=page_id, auth=token_auth, return_raw=False)

domo_page


DomoPage(id='1761849366', title='delete me', parent_page_id=None, top_page_id=None, owners=[DomoUser(id='1893952720', title=None, department='Business Improvement', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=1, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=[DictDot(id=2073919085, title='How did education impact povert', description='', minimized=False, cardIndices=[]), DictDot(id=397530436, title='Whats the ratio between local versus foreign students', description='', minimized=False, cardIndices=[0, 1])], children=[], is_locked=None)

In [73]:
#| exporti

@patch_to(DomoPage,cls_method=True)
async def _from_adminsummary(cls, page_obj, auth :dmda.DomoAuth):

    import domolibrary.classes.DomoCard as dmc

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(
        id=dd.id or dd.pageId,
        title=dd.title or dd.pageTitle,
        parent_page_id=dd.parentPageId,
        top_page_id = dd.topPageId,
        collections=dd.collections,
        is_locked = dd.locked,
        auth=auth,
    )

    if dd.page and dd.page.owners and len(dd.page.owners) > 0:
        pg.owners = await pg._get_domo_users(page_obj.get('page').get('owners'))

    elif dd.owners and len(dd.owners) > 0:
        pg.owners = await pg._get_domo_users(page_obj.get('owners'))


    if dd.cards and len(dd.cards) > 0:
        pg.cards = await asyncio.gather(
            *[dmc.DomoCard.get_from_id(id=card.id, auth=auth) for card in dd.cards])

    return pg


In [61]:
# | export
@patch_to(DomoPage)
async def get_accesslist(self,
                         auth: dmda.DomoAuth = None,
                         is_expand_users: bool = False,
                         return_raw: bool = False,
                         debug_api: bool = False):

    auth = auth or self.auth

    res = await page_routes.get_page_access_list(auth=auth,
                                                is_expand_users=is_expand_users,
                                                page_id=self.id,
                                                debug_api=debug_api
                                                )

    if return_raw:
        return res

    if not res.is_success :
        raise Exception('error getting access list')

    import domolibrary.classes.DomoUser as dmu
    import domolibrary.classes.DomoGroup as dmg
    
    tasks = await asyncio.gather( 
        dmu.DomoUsers.by_id(user_ids=[user.get('id') for user in res.response.get('users')], only_allow_one=False, auth=auth),
        * [ dmg.DomoGroup.get_by_id( group_id = group.get('id'), auth = auth) for group in res.response.get('groups')]
    )
    
    res.response.update({'users': tasks[0], 'groups': tasks[1:]})

    return res.response


#### sample get_accesslist

In [62]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = DomoPage(id=page_id, auth=token_auth)

await domo_page.get_accesslist(is_expand_users=True, return_raw=False, debug_api=False)


{'users': [DomoUser(id='1216550715', title=None, department=None, display_name='8:26 - go to sleep', email_address='test4@domo.com', role_id=2, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
  DomoUser(id='1628021317', title=None, department=None, display_name='Aaron Dean', email_address='aaron.dean@rxa.io', role_id=2097317660, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
  DomoUser(id='1542225148', title=None, department=None, display_name='Adam Landefeld', email_address='Adam.Landefeld@domo.com', role_id=2097317660, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to

In [63]:
#| export

@dataclass
class DomoPages:
    pass

In [64]:
# | exporti

@patch_to(DomoPages, cls_method=True)
async def get_pages(cls: DomoPages, auth=dmda.DomoAuth,
                    return_raw: bool = False,
                    debug_loop: bool = False, debug_api: bool = False, session: httpx.AsyncClient = None):
    is_close_session = False if session else True

    session = session or httpx.AsyncClient()

    try:
        res = await page_routes.get_pages_adminsummary(auth=auth, debug_loop=False, debug_api=False, session=session)

        if return_raw: 
            return res
            
        if not res.is_success:
            raise Exception('unable to retrieve pages')
        

        return await asyncio.gather(*[DomoPage._from_adminsummary(page_obj, auth=auth) for page_obj in res.response])

    finally:
        if is_close_session:
            await session.aclose()


In [74]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoPages.get_pages(auth=token_auth, return_raw=False)


[DomoPage(id=1316566624, title='20210623_TRAINING_DomoStats Activity Log App', parent_page_id=127044793, top_page_id=522373865, owners=[DomoUser(id='1893952720', title=None, department='Business Improvement', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=1, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=None, children=[], is_locked=False),
 DomoPage(id=384424178, title='75th Percentile Test', parent_page_id=None, top_page_id=None, owners=[DomoUser(id='1898323170', title=None, department=None, display_name='Creed Smith', email_address='creed.smith@domo.com', role_id=2, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_

In [66]:


#     @classmethod
#     async def get_cards(cls, auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoCard as dc
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(auth=auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 card_list = [dc.DomoCard(id=card.get(
#                     'id'), auth=auth) for card in json.get('cards')]

#                 return card_list

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

#     async def get_datasets(auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoDataset as dmds
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(auth=auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 dataset_ls = [card.get('datasources')
#                               for card in json.get('cards')]

#                 return [dmds.DomoDataset(id=ds.get('dataSourceId'), auth=auth) for ds_ls in dataset_ls for ds in ds_ls]

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

In [67]:
# | hide
import nbdev

nbdev.nbdev_export()